In [3]:
import os, re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import euclidean_distances, cosine_similarity
from scipy.cluster.hierarchy import ward, dendrogram
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager
from pandas import DataFrame, Series

def clean(instring):
    instring = re.sub(r'~~~START\|.+?\|START~~~', "", instring)
    
    instring = re.sub(r'[a-zA-Z0-9]', "", instring)
    
    # Decide on unwanted characters
    unwanted_chars = ['』','。', '！', '，', '：', '、', '（',
                      '）', '；', '？', '〉', '〈', '」', '「',
                      '『', '“', '”', '!', '"', '#', '$', '%',
                      '&', "'", '(', ')', '*', '+', ',', '-',
                      '.', '/', "《", "》", "·"]
    
    for char in unwanted_chars:
        instring = instring.replace(char, "")
    
    return instring

def textBreak(inputstring):
    # Decide how long each section should be
    divlim = 10000
    
    loops = len(inputstring)//divlim
    
    save = []
    
    for i in range(0, loops):
        save.append(inputstring[i * divlim: (i + 1) * divlim])
    
    return save

info_list = []
title_list = []
author_list = []
era_list = []
genre_list = []
section_number = []
title_author = {}
title_era = {}
title_genre = {}

# Decide whether or not to break the text apart
break_apart = True

# Decide whether or not to normalize

normalize = False

metadata = {}

# Add the name of the metadata file
metadatafile = open("metadata.txt", "r", encoding="utf8")
metadatastring = metadatafile.read()
metadatafile.close()

lines = metadatastring.split("\n")
for line in lines:
    cells = line.split("\t")
    metadata[cells[0]] = cells[1:]

# Here you will set the name of the directory the files are stored in.
for root, dirs, files in os.walk("corpus"):
    for filename in files:
        if filename[0] != ".":
            f = open(root + "/" + filename, "r", encoding="utf8")
            c = f.read()
            f.close()
            c = re.sub("\s+", "", c)
            c = clean(c)
            
            metainfo = metadata[filename[:-4]]
            
            title_author[metainfo[0]] = metainfo[1]
            title_era[metainfo[0]] = metainfo[2]
            title_genre[metainfo[0]] = metainfo[3]
            
            if not break_apart:
                info_list.append(c)
                title_list.append(metainfo[0])
                author_list.append(metainfo[1])
                era_list.append(metainfo[2])
                genre_list.append(metainfo[3])

            else:
                broken_sections = textBreak(c)
                
                info_list.extend(broken_sections)

                title_list.extend([metainfo[0] for i in 
                                   range(0,len(broken_sections))])
                author_list.extend([metainfo[1] for i in 
                                    range(0,len(broken_sections))])
                era_list.extend([metainfo[2] for i in 
                                 range(0,len(broken_sections))])
                genre_list.extend([metainfo[3] for i in 
                                   range(0,len(broken_sections))])
                section_number.extend([i for i in range(0, len(broken_sections))])

# Decide how to construct the vectorizer
vectorizer = CountVectorizer(analyzer="char",ngram_range=(1,1),
                             max_features = 100)

word_count_matrix = vectorizer.fit_transform(info_list)

if normalize:
    vectorizer = CountVectorizer(analyzer="char", ngram_range=(1,1),
                                 max_features=100)
    word_count_matrix=vectorizer.fit_transform(info_list)
    vocab = vectorizer.get_feature_names()

    dense_words = word_count_matrix.toarray()

    corpus_dataframe = DataFrame(dense_words, columns=vocab)

    doclengths = corpus_dataframe.sum(axis=1)

    thousand = Series([1000 for i in range(0,len(doclengths))])

    adjusteddoclengths = thousand.divide(doclengths)

    per_thousand = corpus_dataframe.multiply(adjusteddoclengths, axis = 0)

    print(per_thousand)

    word_count_matrix = per_thousand.as_matrix()

vocab = vectorizer.get_feature_names()

# Decide which similarity measure to use
euc_or_cosine = "euc"
if euc_or_cosine == "euc":
    similarity = euclidean_distances(word_count_matrix)
elif euc_or_cosine == "cos":
    similarity = cosine_similarity(word_count_matrix)
    
linkage_matrix = ward(similarity)

from sys import platform
if platform == "linux" or platform == "linux2":
    print("Sorry, I can't see the appropriate fonts, defaulting to Japanese")
    matplotlib.rc('font', family="TakaoPGothic")
elif platform == "win32" or platform == "win64":
    matplotlib.rc('font', family="SimHei")
elif platform == "darwin":
    matplotlib.rc('font', family='STHeiti')

# Here decide how to label the texts
# you can use title_list, author_list, or genre_list
dendrogram(linkage_matrix, labels=title_list)
plt.title("Textual Relationships")
plt.xlabel("Texts")
plt.ylabel("Distance")
plt.xticks(rotation=60)

# Create a colorization schema
# You can base this on title, author, or genre
color_dict = {"传奇":"red", "小说":"blue", "话本":"magenta"}

plt_info = plt.gca()
tick_labels = plt_info.get_xmajorticklabels()

for tick_label in tick_labels:
    # Get the information from a dictionary to feed to color dictionary
    # You can use title_genre, title_author, or title_era
    genre = title_genre[tick_label.get_text()]
    
    color = color_dict[genre]
    print(tick_label.get_text(), genre, color)
    tick_label.set_color(color)
    
# Show the plot
plt.show()

聊斋志异 话本 magenta
聊斋志异 话本 magenta
聊斋志异 话本 magenta
聊斋志异 话本 magenta
聊斋志异 话本 magenta
聊斋志异 话本 magenta
聊斋志异 话本 magenta
聊斋志异 话本 magenta
聊斋志异 话本 magenta
聊斋志异 话本 magenta
聊斋志异 话本 magenta
聊斋志异 话本 magenta
聊斋志异 话本 magenta
聊斋志异 话本 magenta
聊斋志异 话本 magenta
聊斋志异 话本 magenta
聊斋志异 话本 magenta
聊斋志异 话本 magenta
聊斋志异 话本 magenta
聊斋志异 话本 magenta
聊斋志异 话本 magenta
聊斋志异 话本 magenta
聊斋志异 话本 magenta
聊斋志异 话本 magenta
聊斋志异 话本 magenta
聊斋志异 话本 magenta
聊斋志异 话本 magenta
聊斋志异 话本 magenta
聊斋志异 话本 magenta
聊斋志异 话本 magenta
聊斋志异 话本 magenta
聊斋志异 话本 magenta
聊斋志异 话本 magenta
聊斋志异 话本 magenta
聊斋志异 话本 magenta
聊斋志异 话本 magenta
聊斋志异 话本 magenta
聊斋志异 话本 magenta
三国演义 小说 blue
三国演义 小说 blue
三国演义 小说 blue
三国演义 小说 blue
三国演义 小说 blue
三国演义 小说 blue
三国演义 小说 blue
三国演义 小说 blue
三国演义 小说 blue
三国演义 小说 blue
三国演义 小说 blue
三国演义 小说 blue
三国演义 小说 blue
三国演义 小说 blue
三国演义 小说 blue
三国演义 小说 blue
三国演义 小说 blue
三国演义 小说 blue
三国演义 小说 blue
三国演义 小说 blue
三国演义 小说 blue
三国演义 小说 blue
三国演义 小说 blue
三国演义 小说 blue
三国演义 小说 blue
三国演义 小说 blue
三国演义 小说 blue
三国演义 小说 blue
三国演义 小说 blue
三国演义 小说 blue
三国

//anaconda/lib/python3.5/site-packages/matplotlib/collections.py:590: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == str('face'):
